In [1]:
from setup import D

synth_count = 10
d = D(
    synth_count=synth_count,
    context='opl',
    mode='beat_tracker'
)


In [2]:
from domblar.transformations import transpose, map_note
from domblar.patterns import Patternable

class P(Patternable):
    def __init__(self, notes):
        self.notes = notes

    def get_val(self, beat_count):
        '''
        [0, [1, 2, [3, 4], 5], 6]
        0 1 6 0 2 6 0 3 6 0 5 6 0 1 6 0 2 6 0 4 6 0 5 6
        get_val(7) -> 3
        get_val(19) -> 4
        '''
        cur_list = self.notes
        while isinstance(cur_list, list):
            next_beat_count = beat_count // len(cur_list)
            # TODO: replace with recursive call to get_val of sub_list
            cur_list = cur_list[beat_count % len(cur_list)]
            beat_count = next_beat_count
        return cur_list


class I(P):
    def __init__(self, notes):
        super().__init__(notes)
        self.instrument = None  # ? FIXME: make it a required parameter
        self.amp_ = 1.0
        self.pan_ = 0.0
        self.mix_ = 0.25  # FIXME: duplicate of default value from supercollider
        self.lpf_ = None
        self.lpr_ = None

    def i(self, instrument):
        self.instrument = instrument
        return self

    def transpose(self, n):
        self.notes = transpose(self.notes, n)
        return self

    def evaluate(self):
        return self

    def map(self, indices):
        self.notes = [map_note(indices, n) for n in self.notes]
        return self

    def amp(self, val):
        self.amp_ = val
        return self

    def mute(self):
        self.amp_ = 0.0
        return self

    def pan(self, val):
        self.pan_ = val
        return self

    def mix(self, val):
        self.mix_ = val
        return self

    def lpf(self, val, *, lpr=None):
        self.lpf_ = val
        self.lpr_ = lpr
        return self

class PTri(Patternable):
    def __init__(self, start, finish, beats):
        assert (beats % 2 == 0)
        self.v0 = start
        self.v1 = finish
        self.beats = beats

    def get_val(self, beat_count):
        beat_count %= self.beats
        alpha = beat_count / (self.beats / 2)
        if alpha > 1:
            alpha = 2 - alpha
        return self.v0 + (self.v1 - self.v0) * alpha

class PSaw(Patternable):
    def __init__(self, start, finish, beats):
        self.v0 = start
        self.v1 = finish
        self.beats = beats

    def get_val(self, beat_count):
        beat_count %= self.beats
        alpha = beat_count / self.beats
        return self.v0 + (self.v1 - self.v0) * alpha


In [3]:
# d.rec()
# import time
# time.sleep(1)
from domblar.ed import ED, mos

chroma_steps = 22
# chroma_steps = 12
# chroma_steps = 15
# chroma_steps = 17
d.chroma = ED(chroma_steps)
steps = 7
d.scale = mos(steps, 3, d.chroma)  # porcupine[7]
d.bpm = 130*2
_ = '.'
v = [0, 2, 4, 10, 0]

d[0] <[
    I([v[0], _, _, _])
    .i('marimba')
    .transpose(-4*steps)
    .amp(PSaw(1.5, 0.7, 32))
    .mix(P([0.01, 0.05, 0.1, 0.25]))
    .lpf(PTri(250, 350, 320))
    # .mod(9, 0.2)  # 9 - param index; 0.2 - amount; TODO: period; LFO
    # TODO: modulate from percussion to pitch sound!
    # .mute()
]

d[4] <[
    I([_, v[0], _, _])
    .i('marimba')
    .transpose(-4*steps)
    .amp(PSaw(1.0, 0.5, 32))
    .mix(P([0.01, 0.05, 0.1, 0.25]))
    .lpf(PTri(250, 350, 320))
    # .mod(9, 0.2)  # 9 - param index; 0.2 - amount; TODO: period; LFO
    # TODO: modulate from percussion to pitch sound!
    # .mute()
]

d[5] <[
    I([_, _, v[0], _])
    .i('marimba')
    .transpose(-4*steps)
    .amp(PSaw(0.7, 0.3, 32))
    .mix(P([0.01, 0.05, 0.1, 0.25]))
    .lpf(PTri(250, 350, 320))
    # .mod(9, 0.2)  # 9 - param index; 0.2 - amount; TODO: period; LFO
    # TODO: modulate from percussion to pitch sound!
    # .mute()
]

d[1] <[
    I([v[0]] + [_] * 3)
    .transpose(-2*steps)
    .i('dune_fmsnare')
    # .amp(0.7)
    .amp(PSaw(0.7, 0.0, 32))
    # .amp(0)
    # .lpf(PTri(1000, 6000, 64))
    .lpf(
        # PTri(2500, 2000, 32),
        P([1500, 2500, [850, 900, 800, 1000], 500, 1000, 850, 1200, [500, 750]]),
        lpr=P([1, 0.5, 0.3, 0.25, 0.1]))
    .pan(PTri(-1, 1, 32))
]
d[2] <[
    I([v[1]] + [_] * 3)
    .transpose(-2*steps)
    .i('dune_fmsnare')
    # .amp(0.7)
    .amp(PSaw(0.7, 0.0, 32))
    # .amp(0)
    .lpf(
        # PTri(2500, 2000, 32),
        P([1500, 2500, [850, 900, 800, 1000], 500, 1000, 850, 1200, [500, 750]]),
        lpr=P([1, 0.5, 0.3, 0.25, 0.1]))
    .pan(PTri(-1, 1, 32))
]
d[3] <[
    I([v[2]] + [_] * 3)
    .transpose(-2*steps)
    .i('dune_fmsnare')
    # .amp(0.7)
    .amp(PSaw(0.7, 0.0, 32))
    # .amp(0)
    .lpf(
        # PTri(2500, 2000, 32),
        P([1500, 2500, [850, 900, 800, 1000], 500, 1000, 850, 1200, [500, 750]]),
        lpr=P([1, 0.5, 0.3, 0.25, 0.1]))
    .pan(PTri(-1, 1, 32))
]

d[9] <[
    I([v[0],v[0]+steps,v[0],v[0]-steps])
    # I([v[0]])
    .i('sb1')
    .transpose(-2*steps)
    .amp(P([0.35, 0.24, 0.35, 0.245]))
    .amp(0.0)
    .mix(0.01)
    .lpf(
        # PTri(2500, 2000, 32),
        P([1500, 2500, [850, 900, 800, 1000], 500, 1000, 850, 1200, [500, 750]]),
        lpr=P([1, 0.5, 0.3, 0.25]))
]


events <class '__main__.I'>
[-28, '.', '.', '.']
Queued!
events <class '__main__.I'>
['.', -28, '.', '.']
Queued!
events <class '__main__.I'>
['.', '.', -28, '.']
Queued!
events <class '__main__.I'>
[-14, '.', '.', '.']
Queued!
events <class '__main__.I'>
[-12, '.', '.', '.']
Queued!
events <class '__main__.I'>
[-10, '.', '.', '.']
Queued!
events <class '__main__.I'>
[-14, -7, -14, -21]
Queued!


In [4]:
# d.finetune(0, 2)
d.beat_tracker.stop()


# import time
# time.sleep(2)
# d.stop_rec()

In [13]:
# TODO-s
# ? bug fix: bad scheduling; losing a lot of sounds
# swing
# TODO: param modulation
# how to vary params? as a diff to original value?
#
# TODO: hierarchy of scales+maps
# hold notes
# scale modulations / chord progressions
# arp / arpeggio
# nudge / shift beats for swing/jazz
# lacing patterns
# add more timbres
# add soundgoodizer compressor
# remove magic constants from beat_tracker.py: bpm * 2; sync = 8
# ? increase bass
# add room and damp params from FreeVerb reverb

from domblar.ed import ED, mos

chroma_steps = 22
d.chroma = ED(chroma_steps)
steps = 7
d.scale = mos(steps, 3, d.chroma)  # porcupine[7]
# FIXME: remove chroma, rewrite as hierarchy of scales
# d.scales = [
#     ED(chroma_steps),
#     mos(steps, 3, d.chroma),  # porcupine[7]
#     [0, 4, 6, 10]
# ]

# d.bpm = 135*1.5

_ = '.'
# h = '-'  # TODO: hold/hyphen

# v = [0, -1, -2, 0, 3, 0, 4, 10-steps, 6, 0+2*steps, 0, 4, 6, 11]
v = [0, 2, 4, 10]

d[0] <[
    I([v[0], _, _, _])
    .i('marimba')
    .transpose(-4*steps)
    .amp(2.0)
    .mix(0.01)
    # .amp(P([1, 0.75, 0.5, 0.25]))
    # .amp(P())
    .lpf(PTri(200, 300, 320))
    # .lpf(200)
    # .mod('param', P())
]
d[1] <[
    I([_, v[0], _, _,])
    .i('marimba')
    .transpose(-3*steps)
    .amp(1.5)
    .mix(0.1)
    # .lpf(300)
    .lpf(PTri(300, 200, 320))
    # .amp(0)
    # .lpf(PTri(200, 300, 32))
]
d[2] <[
    I([_, _, v[0], _,])
    .i('marimba')
    .transpose(-3*steps)
    .amp(0.5)
    # .amp(0)
    # .lpf(PTri(100, 3000, 32))
]
d[3] <[
    I([_, _, _, v[0],])
    .i('marimba')
    .transpose(-4*steps)
    .amp(0.25)
    # .amp(0)
    # .lpf(PTri(2000, 300, 32))
]

d[4] <[
    I([v[0], _, _, v[1], v[3],
       v[0], _, _, _, v[1], _, v[2], _,])
    # .i('marimba2')
    .i('sdrum2')
    .amp(1.0)
    .amp(0)
    .lpf(PTri(1000, 10000, 128))
    .pan(PTri(-1, 1, 32))
    .transpose(-steps)
]

d[5] <[
    I([v[0], v[3], v[2],
       v[1], _, _,
    #    v[1], h, h,
       v[3], _, _,
       v[2], _,
       ])
    .transpose(-2*steps)
    .i('dune_fmsnare')
    .amp(0.7)
    # .amp(0)
    .lpf(PTri(1000, 6000, 64))
    .pan(PTri(-1, 1, 32))
]
d[6] <[
    I([v[1], v[2], v[0], _,
       _, v[3], _, _,
       v[2], _, _, _,
       v[0], _, _, _,
       ])
    .transpose(-steps)
    .i('dune_fmsnare')
    .amp(0.7)
    .amp(0)
    .lpf(PTri(1000, 6000, 66))
    .pan(PTri(-1, 1, 34))
]
d[7] <[
    I([3, _, _, _, _, _,
       2, _, _, _, _, _,
       0, _, _, _, _, _,
       1, _, _, _, _, _,
       ])
    .map(v)
    .transpose(-steps)
    # .i('dune_fmsnare')
    .i('string7')
    # .amp(0.7)
    # .mix(0.5)
    .amp(0)
    .lpf(PTri(1000, 6000, 62))
    .pan(PTri(-1, 1, 36))
]
d[8] <[
    I([v[2], _, _, _, _,
       v[0], _, _, _,
       v[1], _, _, _, _,
       v[3], _, _, _, _,
       ])
    # .transpose(-3)
    .i('dune_fmsnare')
    .amp(0.7)
    # .amp(0.0)
    .lpf(PTri(1000, 6000, 68))
    .pan(PTri(-1, 1, 38))
]



# d[5] <[
#     # I([_, 0, 0, 0, _, 0, _, 0])
#     # I([_, 0, _, 0, _, 0, _, 0])
#     # I([5, 3, 0, 2, 1, 4, 7])
#     I([4, _, 2, 0, 2])
#     .i('dune_fmsnare')
#     .amp(0.7)
#     # .amp(0)
#     .lpf(PTri(200, 10000, 64))
#     # abs(math.sin(self.beat_count/8/8*math.pi)) * 2000 + 100
#     # tidal: lpf (range 1000 400 $ slow 32 sine)
#     # foxdot: P[1500, 2500, 850, 500, 1000, [500, 750]]
#     .transpose(-3*steps)
# ]

# d[6] <[
#     I([_, 0, _, _, _, 0, 0, 0, _])
#     .i('string7')
#     .transpose(-1*steps)
#     .amp(0.4)
#     .amp(0)
#     # .lpf(PTri(200, 1000, 32))
#     # .mute()
# ]

# d[2] <[
#     I([0, _, 2, _, 0, _, 1, _,])
#     .i('sdrum2')
#     .transpose(-steps)
#     .amp(0.5)
#     # .mute()
# ]


events <class '__main__.I'>
[-28, '.', '.', '.']
Queued!
events <class '__main__.I'>
['.', -21, '.', '.']
Queued!
events <class '__main__.I'>
['.', '.', -21, '.']
Queued!
events <class '__main__.I'>
['.', '.', '.', -28]
Queued!
events <class '__main__.I'>
[-7, '.', '.', -5, 3, -7, '.', '.', '.', -5, '.', -3, '.']
Queued!
events <class '__main__.I'>
[-14, -4, -10, -12, '.', '.', -4, '.', '.', -10, '.']
Queued!
events <class '__main__.I'>
[-5, -3, -7, '.', '.', 3, '.', '.', -3, '.', '.', '.', -7, '.', '.', '.']
Queued!
events <class '__main__.I'>
[3, '.', '.', '.', '.', '.', -3, '.', '.', '.', '.', '.', -7, '.', '.', '.', '.', '.', -5, '.', '.', '.', '.', '.']
Queued!
events <class '__main__.I'>
[4, '.', '.', '.', '.', 0, '.', '.', '.', 2, '.', '.', '.', '.', 10, '.', '.', '.', '.']
Queued!


In [9]:
d.beat_tracker.stop()

In [23]:

# TODO: more like FoxDot:
# d[0] <[
#     I('marimba'
#       , [0, _, _, _,]
#       , amp=1
#       , lpf=PTri(2000, 100, 320)
#     #   , mod_param=P()
#     )
#     .transpose(-4*steps-1)
#     # .mod('param', P())
#     )
# ]


# FoxDot:
# SynthDef.new(\play2,
# {|amp=1, sus=1, pan=0, freq=0, vib=0, fmod=0, rate=1.0, bus=0, blur=1, beat_dur=1, atk=0.01, decay=0.01, rel=0.01, peak=1, level=0.8, buf=0, pos=0, room=0.1|
# var osc, env;
# sus = sus * blur;
# rate = In.kr(bus, 1);
# osc=PlayBuf.ar(2, buf, (BufRateScale.ir(buf) * rate), startPos: (BufSampleRate.kr(buf) * pos));
# osc=(osc * amp);
# osc = Mix(osc) * 0.5;
# osc = Pan2.ar(osc, pan);
# 	ReplaceOut.ar(bus, osc)}).add;

# self.input     = "osc = In.{}(bus, {});\n".format(self.suffix, self.channels)
# self.output    = "ReplaceOut.{}".format(self.suffix)
# FxList = EffectManager()
# fx = FxList.new('lpf','lowPassFilter', {'lpf': 0, 'lpr': 1}, order=2)
# fx.add('osc = RLPF.ar(osc, lpf, lpr)')
# fx.save()


# # euclidean rhythms
# # tidalcycles:
# # d1 $ s "kick(<13 10 9 11 5 7>, 16, <0 6 10 2>)"

# from domblar.ed import ED, mos

# chroma_steps = 22
# d.chroma = ED(chroma_steps)
# steps = 7
# d.scale = mos(steps, 3, d.chroma)  # porcupine[7]
# d.bpm = 135*2
# _ = '.'
# d[0] <[
#     I([0, _, _, _,])
#     # I([[0, 0], _, _, _,])  # TODO: nested tuplets
#     # I([(0, 2), _, _, _,])  # TODO: lacing
#     # P(euc(3, 8))  # TODO: euclidean rhythms
#     # P[1,2,3] + [3,4] == P[4, 6, 6, 5, 5, 7]
#     # P[1,2,3] * 2 == P[2, 4, 6]  # TODO: (do we need this?)
#     # PRange(10) == P[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
#     # PRange(4) | [1,7,6] == P[0, 1, 2, 3, 1, 7, 6]
#     .i('marimba')
#     .transpose(-4*steps-1)
#     .amp(1)
#     # .mute()
# ]
# d[1] < [
#     # I([_, 0, 0, 0, _, 0, _, 0])
#     # I([_, 0, _, 0, _, 0, _, 0])
#     I([_, _, _, _, 0, _, _, _,
#        _, _, _, _, 0, _, 0, _,])
#     .i('sdrum2')
#     # .amp(1.5)
#     .amp(0)
#     .transpose(-steps)
# ]
# d[2] < [
#     # I([_, 0, 0, 0, _, 0, _, 0])
#     # I([_, 0, _, 0, _, 0, _, 0])
#     I([4, 2, 0, 2])
#     .i('dune_fmsnare')
#     .amp(0.15)
#     .amp(0)
#     .transpose(-3*steps)
# ]
# d[3] <[
#     I([_, 0, _, _, _, 0, 0, 0, _])
#     .i('string7')
#     .transpose(-1*steps)
#     .amp(0.2)
#     .amp(0)
#     # .mute()
# ]
# # d[2] <[
# #     I([0, _, 2, _, 0, _, 1, _,])
# #     .i('sdrum2')
# #     .transpose(-steps)
# #     .amp(0.5)
# #     # .mute()
# # ]


In [323]:
d.finetune(0, 2)
# d.finetune(2, 2)

In [ ]:
# TODO: code/implementation problems:
# ? fix bug: when switching to new patterns, they could sync on different beats
# - send lpf (pan, amp, ...) value separately
# - foxdot: how do they send float values with OSC?
#     - OSC-messages with 'i' (int32), 'f' (float32), 'd' (double), 's' (string) and
#    'b' (blob / binary data) types
# - foxdot: how do they work with processing sound? group, ...

# TODO: alternative notation ideas
# d[1] < 'snare' < P('01..01..')  # 0 - open, 1 - closed?
#
# (meh) d[0] < 'i:kick' < Root(0) < 'p:x.x.x.x.'

# TODO: instruments, opl2
# - kick / bass drum
# - snare
# ...

# TODO: interactive drum machine
# - notation for patterns
# ? 'x.-' notation (no note values)
# - interactivity: ...
# ...

# TODO: opl2 specifics
# - effects
# ...

# TODO: (chiptune) tracker specifics
# ...

# TODO: algorithms
# - mutation
# - swing; jazz swing
# ...

# TODO: mixing/automation/sound design
# - parameter modulation
# - riser/sweep
# - 303 acid (+ portamento)
# ...

# TODO: types of rhythms
# - Toussaint book
# - phasing a-la Steve Reich
# - euclidean rhythms
# - perfectly balanced rhythms
# - nested tuplets
# - explore more: polyrhythms a-la Nik Bärtsch
# - hemiola/polyrhythms a-la Philip Glass
# - rhythm modulation
# ? 21212, RLE rhythms
# - Scamp plus xor
# - Spectral accelerando
# ? gray codes
# - MOS rhythm: https://en.xen.wiki/w/MOS_rhythm
# ...

# TODO: harmonies
# - chord progressions
# - dissonances; phrygian/locrian scales
# - dissonant edos: 11edo, 13edo
# ? non-functional harmonies
# - arpeggios
# ...

# TODO: ?
# - smth similar to micrograd class (build_topo + _backward)
# - randomization
# - performative aspects
# - collect the tips and tricks; analyze/recombine/simplify them

In [6]:
# d.set_synth(0, 'sdrum2')
# notes1 = '.xx.xxxx'
# notes1 = [perc_parse(n) for n in notes1]
# d.set_synth(1, 'bdrum_ok')
# notes2 = 'x...x...'
# notes2 = [perc_parse(n, -20) for n in notes2]
# d.set_synth(2, 'dochat2')
# notes3 = '...x.x..'
# notes3 = [perc_parse(n, -10) for n in notes3]


# d.set_synth(0, 'sdrum2')
# notes1 = '....x..x.x..x..x'
# notes1 = [perc_parse(n) for n in notes1]
# d.set_synth(1, 'bdrum_ok')
# notes2 = 'x.x.x.x.x.x.x.x.'
# notes2 = [perc_parse(n, -20) for n in notes2]
# d.set_synth(2, 'dochat2')
# notes3 = 'x.x.......xx....'
# notes3 = [perc_parse(n, -15) for n in notes3]